In [1]:
import io
import os
import math
import torch
import warnings
from itertools import chain
from dataclasses import dataclass
from tqdm.notebook import tqdm
from collections.abc import Mapping
from datasets import load_dataset
from torch.utils.data.dataset import Dataset
from transformers.data.data_collator import DataCollatorMixin
from transformers import (CONFIG_MAPPING,
                          MODEL_FOR_MASKED_LM_MAPPING,
                          PreTrainedTokenizer,
                          TrainingArguments,
                          AutoConfig,
                          AutoTokenizer,
                          AutoModelForMaskedLM,
                          DataCollatorForLanguageModeling,
                          DataCollatorForWholeWordMask,
                          PretrainedConfig,
                          Trainer,
                          set_seed)

# Set seed for reproducibility,
set_seed(69)
#chay lai notebook cung ket qua

# Look for gpu to use. Will use `cpu` by default if no gpu found.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [2]:
device

device(type='cpu')

In [3]:
import pandas as pd

# Load the dataset to examine its structure
data_path = 'shuffled_quarter_final.csv'
data = pd.read_csv(data_path)
data.head(30)


,content,label
0,"Giờ đu công, chui thùng lạnh thôi. Đi XKLĐ cũn...",0
1,TẬP CẬN BÌNH QUA VIỆT NAM THĂM LÀ ĐIỀU RẤT TỐT...,0
2,Có phải “ chiên anton “ là thằng hôm trước về ...,0
3,Hay quá 🌷💙Buổi nói chuyện của 2 vị rất hay. Xi...,0
4,Sao không đòi công lý cho nạn nhân bị nhiễm ch...,0
5,quân trương phân tích lịch sử rất sâu să...,0
6,"Chxhcn việt nam (trừ tự do, trừ hạnh phúc)😂😂😂",0
7,Ngu vậy sao giàu được,0
8,Đây chắc là cơ hội cho đang dân chủ thằng chiế...,0
9,Vin cũng biết tận dụng bán hàng,0


In [5]:
import unicodedata
import pandas as pd
import regex as re
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as f:
        black_words = f.read().splitlines()

def remove_control_and_normalize(text):

    text = ''.join(char for char in text if not unicodedata.combining(char))

    text = re.sub(r'(https?://[^\s]+)', 'URL', text)

    text = unicodedata.normalize('NFKC', text)
    text = text.replace('/', ' ')
    text = text.replace('(', ' ')
    text = text.replace(')', ' ')
    text = re.sub(r'\b\w*\d\w*\b', '', text)  # Loại bỏ từ có số

    text = re.sub(r'[^\w\s]', '', text, flags=re.UNICODE)

    # text = text.lower()


    text = re.sub(r'\s+', ' ', text).strip()


    # text = " ".join(word for word in text.split() if word not in black_words)


    return text


In [6]:
data['cleaned_content'] = data['content'].apply(remove_control_and_normalize)

# Hiển thị một số dòng đầu để kiểm tra kết quả
data[['content', 'cleaned_content']].head(30)


,content,cleaned_content
0,"Giờ đu công, chui thùng lạnh thôi. Đi XKLĐ cũn...",Giờ đu công chui thùng lạnh thôi Đi XKLĐ cũng ...
1,TẬP CẬN BÌNH QUA VIỆT NAM THĂM LÀ ĐIỀU RẤT TỐT...,TẬP CẬN BÌNH QUA VIỆT NAM THĂM LÀ ĐIỀU RẤT TỐT...
2,Có phải “ chiên anton “ là thằng hôm trước về ...,Có phải chiên anton là thằng hôm trước về VN l...
3,Hay quá 🌷💙Buổi nói chuyện của 2 vị rất hay. Xi...,Hay quá Buổi nói chuyện của vị rất hay Xin cảm...
4,Sao không đòi công lý cho nạn nhân bị nhiễm ch...,Sao không đòi công lý cho nạn nhân bị nhiễm ch...
5,quân trương phân tích lịch sử rất sâu să...,quân trương phân tich lich sư rât sâu săc dê hiêu
6,"Chxhcn việt nam (trừ tự do, trừ hạnh phúc)😂😂😂",Chxhcn việt nam trừ tự do trừ hạnh phúc
7,Ngu vậy sao giàu được,Ngu vậy sao giàu được
8,Đây chắc là cơ hội cho đang dân chủ thằng chiế...,Đây chắc là cơ hội cho đang dân chủ thằng chiế...
9,Vin cũng biết tận dụng bán hàng,Vin cũng biết tận dụng bán hàng


In [10]:
from tokenizers import models, pre_tokenizers, decoders, trainers, processors, Tokenizer, ByteLevelBPETokenizer

# Initialize a tokenizer with BPE model
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

# Define vocabulary size for BPE
vocab_size = 30000

# Configure the BPE trainer
trainer = trainers.BpeTrainer(
    vocab_size=vocab_size, 
    min_frequency=2, 
    special_tokens=[
        "<s>",  # Start of Sentence
        "<pad>",  # Padding
        "</s>",  # End of Sentence
        "<unk>",  # Unknown Token
        "<mask>",  # Masking Token
    ]
)

# Train the tokenizer using the cleaned content
cleaned_content = data['cleaned_content']
tokenizer.train_from_iterator(iterator=cleaned_content, trainer=trainer)
tokenizer.save("tokenizer.json")

# Set up a RoBERTa-style post-processor


In [11]:
tokenizer.post_processor = processors.RobertaProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)

# Enable truncation to a maximum length of 512 tokens
tokenizer.enable_truncation(max_length=512)

# Save or return the tokenizer for further use

encoding = tokenizer.encode("Tao là bố mày!")
encoding.tokens

['<s>', 'T', 'ao', 'ĠlÃł', 'Ġbá»ĳ', 'ĠmÃły', '</s>']

In [12]:
tokenizer.decoder = decoders.ByteLevel()
tokenizer.decode(encoding.ids)

'Tao là bố mày'

In [3]:
# Import thư viện cần thiết
from transformers import RobertaConfig, RobertaModel,RobertaForMaskedLM, RobertaTokenizerFast, DataCollatorForLanguageModeling, Trainer, TrainingArguments, AutoTokenizer, CONFIG_MAPPING
from datasets import load_dataset

# Thiết lập cấu hình mô hình với thông số tùy chỉnh
model_type = 'roberta'  # Loại mô hình
model_config = CONFIG_MAPPING[model_type]()  # Cấu hình mặc định cho mô hình
override_config = {
    "num_hidden_layers": 8,
    "num_attention_heads": 8,
    "intermediate_size": 2048,
    "hidden_size": 512,
    "max_position_embeddings": 130,  # 128 cho nội dung + 2 cho token đặc biệt
}
model_config.update(override_config)  # Cập nhật cấu hình tùy chỉnh

# Khởi tạo mô hình RoBERTa với cấu hình tùy chỉnh
model = RobertaForMaskedLM(model_config)

# Sử dụng tokenizer đã huấn luyện từ tệp hoặc thư mục
tokenizer_path = "C:\\Users\\Admin\\Downloads\\roberta\\mlm_bert_2"  # Đường dẫn đến thư mục chứa tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# Sử dụng bộ dữ liệu CSV đã cho
dataset_path = "cleaned_quarter_final.csv"
dataset = load_dataset("csv", data_files={"train": dataset_path})
train_dataset = dataset["train"]
for i in range(len(train_dataset)):
    content = train_dataset[i].get("cleaned_content", None)
    
    if content is None or not isinstance(content, str):
        print(f"Element {i} is invalid: {content}")  # Ghi lại phần tử lỗi

# Hàm kiểm tra và chuyển đổi dữ liệu thành chuỗi
# Hàm để đảm bảo mọi giá trị là chuỗi và trả về danh sách
# for i in range(999 ,1005 ):
#     content = train_dataset[i].get("cleaned_content", "")
#     if content is None:
#         print(f"Data {i} is None or empty.")
# Kiểm tra sau khi lọc
# Lọc các phần tử có giá trị None trong "cleaned_content"
valid_dataset = train_dataset.filter(lambda example: example.get("cleaned_content") is not None)
# Loại bỏ cột label
dataset_without_label = valid_dataset.remove_columns("label") 
# Hoặc tên cột liên quan đến nhãn
dataset_without_label = dataset_without_label.remove_columns("content")
# Tokenize dữ liệu và thiết lập Data Collator cho Language Modeling
def tokenize_function(example):
    content = example.get("cleaned_content", "")
    
    # Nếu là None hoặc không phải là chuỗi, chuyển thành chuỗi
    if content is None or not isinstance(content, str):
        content = ""

    return tokenizer(example["cleaned_content"], truncation=True, max_length=130, padding="max_length")

tokenized_dataset = dataset_without_label.map(tokenize_function, batched=True)
# Lấy 5 phần tử đầu tiên
# Lấy 5 phần tử đầu tiên từ tập dữ liệu
first_5_examples = tokenized_dataset.select(range(5))

for example in first_5_examples:
    print("Tokenized content:", example["input_ids"])  # Kiểm tra nội dung được token hóa

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,  # Masked Language Modeling
    mlm_probability=0.15  # Tỷ lệ phần trăm các từ được mask
)

# Thiết lập các tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_strategy="steps",
    save_steps=500,
    logging_dir="./logs",
    logging_steps=500,
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_steps=500,
)

# Thiết lập Trainer và thực hiện huấn luyện
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Thực hiện pre-training mô hình
trainer.train()


Element 1471 is invalid: None
Element 1894 is invalid: None


The following columns in the training set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: cleaned_content. If cleaned_content are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
c:\Users\Admin\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4797
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1800
  Number of trainable parameters = 41209146


Tokenized content: [5775, 1209, 425, 2435, 4403, 2993, 468, 312, 204, 42, 43, 661, 324, 3567, 2539, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Tokenized content: [51, 466, 3322, 680, 420, 664, 511, 2092, 187, 896, 457, 758, 1204, 2447, 721, 507, 457, 846, 866, 664, 511, 3537, 1201, 3564, 587, 420, 664, 511, 721, 232, 231, 420, 260, 5994, 893, 1818, 420, 664, 511, 468, 613, 411, 231, 1023, 2192, 284, 461, 794, 2491, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Tokenized content: [2816, 334, 3871,

  0%|          | 0/1800 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 